# 📦 Importing Required Libraries

In this section, we import all necessary Python libraries used throughout the project.

These libraries are responsible for:

- **Data manipulation:** `pandas`, `numpy`
- **Visualization:** `matplotlib`
- **Mathematical operations:** `math`

Keeping all imports at the beginning of the notebook improves readability and reproducibility.


In [6]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt

# Mathematical operations
from math import sqrt

# Optional: Improve plot aesthetics
plt.style.use("seaborn-v0_8")

# 📂 Loading the Dataset

In this section, we load the datasets required for building the recommendation system.

We use two CSV files:

- `movies.csv` → Contains movie metadata (movieId, title, genres)
- `ratings.csv` → Contains user ratings for movies

These datasets are provided by GroupLens (IMDB-based dataset).

After loading the data, we will inspect their structure.


In [ ]:
# Define dataset paths (recommended for better project structure)
MOVIES_PATH = "data/movies.csv"
RATINGS_PATH = "data/ratings.csv"

# Load datasets into pandas DataFrames
movies_df = pd.read_csv(MOVIES_PATH)
ratings_df = pd.read_csv(RATINGS_PATH)

# Display basic information
print("Movies Dataset Shape:", movies_df.shape)
print("Ratings Dataset Shape:", ratings_df.shape)

Movies Dataset Shape: (9742, 3)
Ratings Dataset Shape: (100836, 4)


# 🔎 Initial Data Exploration (Movies Dataset)

In this section, we perform an initial exploration of the `movies` dataset to better understand its structure.

We will:

- Preview the first few rows
- Check dataset dimensions
- Inspect data types
- Generate statistical summaries (where applicable)

This step helps us understand the structure and quality of the dataset before preprocessing.

In [7]:
# Initial Exploration - Movies Dataset
print("First 5 Rows:")
display(movies_df.head())

print("\n Dataset Shape:")
print(movies_df.shape)

print("\n Data Types:")
display(movies_df.dtypes)

print("\n Statistical Summary:")
display(movies_df.describe(include='all'))


First 5 Rows:


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy



 Dataset Shape:
(9742, 3)

 Data Types:


movieId     int64
title      object
genres     object
dtype: object


 Statistical Summary:


,movieId,title,genres
count,9742.000000,9742,9742
unique,NaN,9737,951
top,NaN,Emma (1996),Drama
freq,NaN,2,1053
mean,42200.353623,NaN,NaN
std,52160.494854,NaN,NaN
min,1.000000,NaN,NaN
25%,3248.250000,NaN,NaN
50%,7300.000000,NaN,NaN
75%,76232.000000,NaN,NaN


# 🧹 Preprocessing Movies Data

In this step, we clean and preprocess the `movies` dataset to make it ready for modeling.

We perform the following transformations:

1. **Extract release year** from the movie title and store it in a new column `year`
2. **Clean movie titles** by removing the year part (e.g., "(1995)") and trimming extra spaces
3. **Parse genres** by splitting the `genres` string into a list of genre labels

These features will be useful for:
- Content-based recommendation (genres)
- Better similarity calculations (clean titles)
- Optional clustering/analysis by release year


In [8]:
# 1) Extract release year from title (4-digit year inside parentheses)
movies_df["year"] = movies_df["title"].str.extract(r"\((\d{4})\)")
movies_df["year"] = pd.to_numeric(movies_df["year"], errors="coerce")

# 2) Clean title by removing the year and extra spaces
movies_df["title"] = movies_df["title"].str.replace(r"\(\d{4}\)", "", regex=True).str.strip()

# 3) Convert genres from "Action|Comedy|..." to a list of genres
movies_df["genres"] = movies_df["genres"].str.split("|")

# Quick sanity-check
display(movies_df.head())


,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995.0
4,5,Father of the Bride Part II,[Comedy],1995.0


# 🎬 Genre Encoding (One-Hot Representation)

To use genres in similarity calculations and recommendation models, 
we convert the list of genres into a One-Hot encoded format.

Each genre becomes a separate binary column:

- 1 → Movie belongs to that genre
- 0 → Movie does not belong to that genre

This transformation is essential for:
- Content-based filtering
- User preference profiling
- Clustering users based on genre interests

In [9]:
# Make a copy of the dataset
movies_with_genres_df = movies_df.copy()

# Explode genres list into separate rows
exploded_df = movies_with_genres_df.explode("genres")

# Apply one-hot encoding
genres_dummies = pd.get_dummies(exploded_df["genres"])

# Combine back with original movie info
movies_with_genres_df = (
    exploded_df[["movieId", "title", "year"]]
    .join(genres_dummies)
    .groupby(["movieId", "title", "year"], as_index=False)
    .sum()
)

# Preview result
display(movies_with_genres_df.head())

# Show genre columns
print("Genre Columns:")
display(movies_with_genres_df.columns[3:])

,movieId,title,year,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story,1995.0,0,0,5,5,5,5,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,1995.0,0,0,3,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men,1995.0,0,0,0,0,0,2,0,...,0,0,0,0,0,2,0,0,0,0
3,4,Waiting to Exhale,1995.0,0,0,0,0,0,3,0,...,0,0,0,0,0,3,0,0,0,0
4,5,Father of the Bride Part II,1995.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


Genre Columns:


Index(['(no genres listed)', 'Action', 'Adventure', 'Animation', 'Children',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
       'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',
       'War', 'Western'],
      dtype='object')

# ⭐ Preprocessing Ratings Data

In this step, we clean and prepare the `ratings` dataset.

The dataset contains:

- `userId`
- `movieId`
- `rating`
- `timestamp`

Since the timestamp is not required for our current recommendation approach,
we remove it to simplify the dataset.

Later, this dataset will be used to:
- Build user profiles
- Construct the user-item interaction matrix
- Perform collaborative filtering


In [10]:
# Preview first rows
print(" Initial Ratings Data:")
display(ratings_df.head())

# Remove timestamp column (not needed for current analysis)
ratings_df = ratings_df.drop(columns=["timestamp"])

# Check structure after modification
print("\n Ratings Data After Dropping 'timestamp':")
display(ratings_df.head())

print("\n Dataset Shape:")
print(ratings_df.shape)

 Initial Ratings Data:


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931



 Ratings Data After Dropping 'timestamp':


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0



 Dataset Shape:
(100836, 3)


# 👤 Simulating a New User (Cold-Start Input)

To demonstrate a **content-based recommendation** workflow, we create a small set of movies rated by a hypothetical new user.

This is a typical *cold-start* scenario where:
- The user has no history in the ratings dataset
- We only know a few rated movies from them

Next, we map the input movie titles to their corresponding `movieId` in `movies_df`,
then merge the ratings into a single dataframe for further processing.


In [12]:
user_input = [
    {'title' : 'Brreakfast Club, The', 'rating':5},
    {'title' : 'Toy Story', 'rating':3.5},
    {'title' : 'Jumanji', 'rating':2},
    {'title' : 'Pulp Fiction', 'rating':5},
    {'title' : 'Akira', 'rating':4.5}

]

In [13]:
input_movies = pd.DataFrame(user_input)

print(" User Input (raw):")
display(input_movies)

# Step 2) Map titles to movieId (exact-title match)
matched_movies = movies_df[movies_df["title"].isin(input_movies["title"])][["movieId", "title", "genres", "year"]]

# Step 3) Merge user ratings with movie metadata
input_movies = matched_movies.merge(input_movies, on="title", how="right")

# Step 4) Warn if some titles were not matched
missing_titles = input_movies[input_movies["movieId"].isna()]["title"].tolist()
if missing_titles:
    print(" Warning: These titles were not found in movies_df (check spelling):")
    for t in missing_titles:
        print("-", t)

print("\n User Input After Mapping to movieId:")
display(input_movies)

 User Input (raw):


,title,rating
0,"Brreakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


- Brreakfast Club, The

 User Input After Mapping to movieId:


,movieId,title,genres,year,rating
0,NaN,"Brreakfast Club, The",NaN,NaN,5.0
1,1.0,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995.0,3.5
2,2.0,Jumanji,"[Adventure, Children, Fantasy]",1995.0,2.0
3,296.0,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994.0,5.0
4,1274.0,Akira,"[Action, Adventure, Animation, Sci-Fi]",1988.0,4.5


# 🧹 Cleaning User Input Data

After merging the user ratings with movie metadata, 
we remove unnecessary columns (`genres`, `year`) 
to keep only the essential information required for building the user profile.

Final columns needed:
- `movieId`
- `title`
- `rating`

This cleaned dataframe will be used to calculate the user's genre preference vector.

In [14]:
print(" Columns Before Cleaning:")
display(input_movies.columns)

# Keep only necessary columns for recommendation
input_movies = input_movies.drop(columns=["genres", "year"])

print("\n Cleaned User Input:")
display(input_movies)

 Columns Before Cleaning:


Index(['movieId', 'title', 'genres', 'year', 'rating'], dtype='object')


 Cleaned User Input:


,movieId,title,rating
0,NaN,"Brreakfast Club, The",5.0
1,1.0,Toy Story,3.5
2,2.0,Jumanji,2.0
3,296.0,Pulp Fiction,5.0
4,1274.0,Akira,4.5


# 🎯 Extracting User-Rated Movies (Genre Matrix)

To build the user profile, we first retrieve the one-hot encoded genre representation 
of the movies rated by the user.

We filter the `movies_with_genres_df` dataset using the selected `movieId`s 
from the user's input.

This dataset will be used to compute the user's genre preference vector.

In [15]:
# Filter movies that the user has rated
user_movies = movies_with_genres_df[
    movies_with_genres_df["movieId"].isin(input_movies["movieId"])
].reset_index(drop=True)

print(" User Rated Movies (One-Hot Genre Representation):")
display(user_movies)

 User Rated Movies (One-Hot Genre Representation):


,movieId,title,year,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story,1995.0,0,0,5,5,5,5,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,1995.0,0,0,3,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
2,296,Pulp Fiction,1994.0,0,0,0,0,0,4,4,...,0,0,0,0,0,0,0,4,0,0
3,1274,Akira,1988.0,0,4,4,4,0,0,0,...,0,0,0,0,0,0,4,0,0,0


In [241]:
user_genre_table = usermovies.drop(columns=['movieId', 'genres', 'year', 'title'])

In [242]:
user_genre_table

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [243]:
inputmovies['rating']

0    3.5
1    2.0
2    5.0
3    4.5
Name: rating, dtype: float64

In [244]:
inputmovies.shape

(4, 3)

In [245]:
inputmovies['rating']

0    3.5
1    2.0
2    5.0
3    4.5
Name: rating, dtype: float64

In [246]:
userprofile = user_genre_table.transpose().dot(inputmovies['rating'])

In [247]:
userprofile

Adventure             10.0
Animation              8.0
Children               5.5
Comedy                 8.5
Fantasy                5.5
Romance                0.0
Drama                  5.0
Action                 4.5
Crime                  5.0
Thriller               5.0
Horror                 0.0
Mystery                0.0
Sci-Fi                 4.5
War                    0.0
Musical                0.0
Documentary            0.0
IMAX                   0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

In [250]:
genre_table = movies_with_genres_df.set_index(movies_with_genres_df['movieId'])
genre_table = genre_table.drop(columns=['movieId', 'genres', 'year', 'title'])
genre_table.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [253]:
recommendation_table_df = ((genre_table * userprofile).sum(axis=1)/userprofile.sum())

In [254]:
recommendation_table_df

movieId
1         0.609756
2         0.341463
3         0.138211
4         0.219512
5         0.138211
            ...   
193581    0.430894
193583    0.357724
193585    0.081301
193587    0.203252
193609    0.138211
Length: 9742, dtype: float64

In [257]:
recommendation_table_df = recommendation_table_df.sort_values(ascending=False)
recommendation_table_df.head()

movieId
2987      0.691057
134853    0.691057
108932    0.682927
26340     0.682927
130520    0.682927
dtype: float64

In [258]:
movies_df.loc[movies_df['movieId'].isin(recommendation_table_df.head(20).keys())]

,movieId,title,genres,year
478,546,Super Mario Bros.,"[Action, Adventure, Children, Comedy, Fantasy,...",1993
559,673,Space Jam,"[Adventure, Animation, Children, Comedy, Fanta...",1996
2250,2987,Who Framed Roger Rabbit?,"[Adventure, Animation, Children, Comedy, Crime...",1988
4348,6350,Laputa: Castle in the Sky (Tenkû no shiro Rapy...,"[Action, Adventure, Animation, Children, Fanta...",1986
4631,6902,Interstate 60,"[Adventure, Comedy, Drama, Fantasy, Mystery, S...",2002
5490,26340,"Twelve Tasks of Asterix, The (Les douze travau...","[Action, Adventure, Animation, Children, Comed...",1976
5819,32031,Robots,"[Adventure, Animation, Children, Comedy, Fanta...",2005
6047,40339,Chicken Little,"[Action, Adventure, Animation, Children, Comed...",2005
6448,51939,TMNT (Teenage Mutant Ninja Turtles),"[Action, Adventure, Animation, Children, Comed...",2007
6455,52287,Meet the Robinsons,"[Action, Adventure, Animation, Children, Comed...",2007
